In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pprint import pprint
from tqdm import tqdm, notebook
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import shutil

In [6]:
path = r'C:\Users\재욱\Downloads\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)
country_url = "https://www.brandsoftheworld.com/logos/countries" 
driver.get(country_url)

xpath = '//*[@id="primaryInner"]/div/ul'
division = driver.find_element_by_xpath(xpath)
country_list = division.find_elements_by_tag_name('a') # List형

country_url_list = []
for country_url in country_list:
    country_url_list.append(country_url.get_attribute('href'))

In [3]:
# dict_status 초기화

dict_status = {}
with open("brands_of_the_world.json", "w") as upload_file:
    json.dump(dict_status, upload_file)

## request.urlopen을 사용한 save_images함수

In [ ]:
def save_images(country_name, image_url_list, name_list):
    
    img_folder_path = f'./brands_of_the_world/{country_name}_company_logo/'    
    os.makedirs(img_folder_path, exist_ok=True)
    
    for i, image_url in enumerate(image_url_list):
        try:
            with urllib.request.urlopen(image_url.get_attribute('src')) as f:
                with open(img_folder_path + name_list[i].text +'.jpg','wb') as h:
                    image = f.read()
                    h.write(image)
                    
        except Exception as e:
            print(e)

## requests.get 을 사용한 save_images함수 (joowon)

In [11]:
def save_images(country_name, image_url_list, name_list):
    
    img_folder_path = f'./brands_of_the_world/{country_name}_company_logo/'    
    os.makedirs(img_folder_path, exist_ok=True)
    
    for i, image_url in enumerate(image_url_list):
        try:
            response = requests.get(image_url.get_attribute('src'), stream=True)
            with open(img_folder_path + name_list[i].text +'.jpg','wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
                    
        except Exception as e:
            print(e)

# Old Code

In [13]:
try:
    XPATH_COUNTRY_TABLE = '//*[@id="primaryInner"]/div/div[1]'
    XPATH_COUNTRY_NAME = '//*[@id="primary"]/h1/span'
    print("지난 status 상황 : ")
    with open("brands_of_the_world.json", "r") as download_file:
        dict_status = json.loads(download_file.read())
    pprint(dict_status)
    print('+-----------------------------------------------------------------+')

    # COUNTRY_NUMBER 대신 enumerate를 쓰는 연습
    for idx , country_path in enumerate(notebook.tqdm(country_url_list, desc='iterate range 100')):
        driver.get(country_path)
        country_name = driver.find_element_by_xpath(XPATH_COUNTRY_NAME).text
        
        # 해당 국가의 key-value가 없을 경우. (해당 국가 페이지 처음 방문)
        if country_name not in dict_status.keys():
            dict_status[country_name] = {"status" : 'NOT DONE', "latest_page" : -1} # 현재 국가 save status 초기화
            page_num = 0
        # 해당 국가의 key-value가 있을 경우. DONE이든 NOT DONE이든 (해당 국가 페이지 방문한 적 있음)
        else: 
            page_num = dict_status[country_name]["latest_page"]
            
        # 해당 국가의 value의 status key가 'NOT DONE'일 경우(해당 국가 페이지 쭉 방문해야함.)    
        if dict_status[country_name]["status"] != 'DONE':   
            
            while True:
                driver.get(f'{country_path}?page={page_num}') #f string 사용!
                division = driver.find_element_by_xpath(XPATH_COUNTRY_TABLE)
                name_list = division.find_elements_by_tag_name('li') # List형
                image_url_list = division.find_elements_by_tag_name('img') # 이미지 URL
                
                if len(image_url_list) == 0:
                    break
                    
                save_images(country_name, image_url_list, name_list)
                
                print(f"{country_name}의 {page_num}번째 page 이미지 저장 완료")
                dict_status[country_name]["latest_page"] +=1
                page_num +=1
                    
            dict_status[country_name]["status"] = 'DONE'
            with open("brands_of_the_world.json", "w") as upload_file:
                json.dump(dict_status, upload_file)
            pprint(dict_status)
            print(f'{idx}번째 나라 {country_name}의 모든 이미지 저장 완료')
            print('+-----------------------------------------------------------------+')
         
except Exception as e:
    print(e)
except KeyboardInterrupt:
    print("[KeyboardInterrupt] 현재까지의 status :")
    with open("brands_of_the_world.json", "w") as upload_file:
        json.dump(dict_status, upload_file)
    pprint(dict_status)
    print("\n")
except:
    print('Other error')

지난 status 상황 : 
{'Afghanistan': {'latest_page': 1, 'status': 'DONE'},
 'Albania': {'latest_page': 15, 'status': 'DONE'},
 'Algeria': {'latest_page': 9, 'status': 'DONE'},
 'American Samoa': {'latest_page': 1, 'status': 'DONE'},
 'Andorra': {'latest_page': 1, 'status': 'DONE'},
 'Angola': {'latest_page': 4, 'status': 'DONE'},
 'Anguilla': {'latest_page': 0, 'status': 'DONE'},
 'Antarctica': {'latest_page': 0, 'status': 'DONE'},
 'Antigua and Barbuda': {'latest_page': 0, 'status': 'DONE'},
 'Argentina': {'latest_page': 65, 'status': 'NOT DONE'}}
+-----------------------------------------------------------------+


iterate range 100:   0%|          | 0/240 [00:00<?, ?it/s]

Argentina의 65번째 page 이미지 저장 완료
[Errno 22] Invalid argument: './brands_of_the_world/Argentina_company_logo/Erico Pereyra | Diseño Integral.jpg'
Argentina의 66번째 page 이미지 저장 완료
Argentina의 67번째 page 이미지 저장 완료
Argentina의 68번째 page 이미지 저장 완료
Argentina의 69번째 page 이미지 저장 완료
Argentina의 70번째 page 이미지 저장 완료
Argentina의 71번째 page 이미지 저장 완료
[Errno 22] Invalid argument: './brands_of_the_world/Argentina_company_logo/Peña Oficial "Amigos Fesa".jpg'
Argentina의 72번째 page 이미지 저장 완료
Argentina의 73번째 page 이미지 저장 완료
Argentina의 74번째 page 이미지 저장 완료
Argentina의 75번째 page 이미지 저장 완료
Argentina의 76번째 page 이미지 저장 완료
[Errno 22] Invalid argument: './brands_of_the_world/Argentina_company_logo/DialOne*.jpg'
Argentina의 77번째 page 이미지 저장 완료
Argentina의 78번째 page 이미지 저장 완료
Argentina의 79번째 page 이미지 저장 완료
Argentina의 80번째 page 이미지 저장 완료
[Errno 22] Invalid argument: './brands_of_the_world/Argentina_company_logo/GIG* | División Aromarketing.jpg'
[Errno 2] No such file or directory: './brands_of_the_world/Argentina_company_logo/Caro

Australia의 18번째 page 이미지 저장 완료
Australia의 19번째 page 이미지 저장 완료
Australia의 20번째 page 이미지 저장 완료
Australia의 21번째 page 이미지 저장 완료
Australia의 22번째 page 이미지 저장 완료
Australia의 23번째 page 이미지 저장 완료
Australia의 24번째 page 이미지 저장 완료
Australia의 25번째 page 이미지 저장 완료
Australia의 26번째 page 이미지 저장 완료
Australia의 27번째 page 이미지 저장 완료
Australia의 28번째 page 이미지 저장 완료
Australia의 29번째 page 이미지 저장 완료
Australia의 30번째 page 이미지 저장 완료
Australia의 31번째 page 이미지 저장 완료
Australia의 32번째 page 이미지 저장 완료
Australia의 33번째 page 이미지 저장 완료
Australia의 34번째 page 이미지 저장 완료
Australia의 35번째 page 이미지 저장 완료
[KeyboardInterrupt] 현재까지의 status :
{'Afghanistan': {'latest_page': 1, 'status': 'DONE'},
 'Albania': {'latest_page': 15, 'status': 'DONE'},
 'Algeria': {'latest_page': 9, 'status': 'DONE'},
 'American Samoa': {'latest_page': 1, 'status': 'DONE'},
 'Andorra': {'latest_page': 1, 'status': 'DONE'},
 'Angola': {'latest_page': 4, 'status': 'DONE'},
 'Anguilla': {'latest_page': 0, 'status': 'DONE'},
 'Antarctica': {'latest_page': 0, 'status': '

## read_json 그리고 write_json 함수화

In [14]:
def read_json(file_path):
    with open(file_path, "r") as fp:
        data = json.loads(fp.read())
    return data

def write_json(data, file_path):
    with open(file_path, "w") as fp:
        json.dump(data, fp)
        
# 보통 file pointer -> fp 사용

# FINAL Code

In [15]:
# 빈 딕셔너리 (dict_status) 선언 후 JSON 파일 먼저 만들기
try:
    XPATH_COUNTRY_TABLE = '//*[@id="primaryInner"]/div/div[1]'
    XPATH_COUNTRY_NAME = '//*[@id="primary"]/h1/span'
    print("지난 status 상황 : ")
    dict_status = read_json("brands_of_the_world.json")
    pprint(dict_status)
    print('+-----------------------------------------------------------------+')

    # COUNTRY_NUMBER 대신 enumerate를 쓰는 연습
    for idx , country_path in enumerate(notebook.tqdm(country_url_list, desc='iterate range 100')):
        driver.get(country_path)
        country_name = driver.find_element_by_xpath(XPATH_COUNTRY_NAME).text
        
        # 해당 국가의 key-value가 없을 경우. (해당 국가 페이지 처음 방문)
        if country_name not in dict_status.keys():
            dict_status[country_name] = {"status" : 'NOT DONE', "latest_page" : -1} # 현재 국가 save status 초기화
            page_num = 0
        # 해당 국가의 key-value가 있을 경우. DONE이든 NOT DONE이든 (해당 국가 페이지 방문한 적 있음)
        else: 
            page_num = dict_status[country_name]["latest_page"]
            
        # 해당 국가의 value의 status key가 'NOT DONE'일 경우(해당 국가 페이지 쭉 방문해야함.)    
        if dict_status[country_name]["status"] != 'DONE':   
            
            while True:
                driver.get(f'{country_path}?page={page_num}') #f string 사용!
                division = driver.find_element_by_xpath(XPATH_COUNTRY_TABLE)
                name_list = division.find_elements_by_tag_name('li') # List형
                image_url_list = division.find_elements_by_tag_name('img') # 이미지 URL
                
                if len(image_url_list) == 0:
                    break
                    
                save_images(country_name, image_url_list, name_list)
                
                print(f"{country_name}의 {page_num}번째 page 이미지 저장 완료")
                dict_status[country_name]["latest_page"] +=1
                page_num +=1
                    
            dict_status[country_name]["status"] = 'DONE'
            write_json(dict_status, "brands_of_the_world.json")
            pprint(dict_status)
            print(f'{idx}번째 나라 {country_name}의 모든 이미지 저장 완료')
            print('+-----------------------------------------------------------------+')
         
except Exception as e:
    print(e)
except KeyboardInterrupt:
    print("[KeyboardInterrupt] 현재까지의 status :")
    with open("brands_of_the_world.json", "w") as upload_file:
        json.dump(dict_status, upload_file)
    pprint(dict_status)
    print("\n")
except:
    print('Other error')

지난 status 상황 : 
{'Afghanistan': {'latest_page': 1, 'status': 'DONE'},
 'Albania': {'latest_page': 15, 'status': 'DONE'},
 'Algeria': {'latest_page': 9, 'status': 'DONE'},
 'American Samoa': {'latest_page': 1, 'status': 'DONE'},
 'Andorra': {'latest_page': 1, 'status': 'DONE'},
 'Angola': {'latest_page': 4, 'status': 'DONE'},
 'Anguilla': {'latest_page': 0, 'status': 'DONE'},
 'Antarctica': {'latest_page': 0, 'status': 'DONE'},
 'Antigua and Barbuda': {'latest_page': 0, 'status': 'DONE'},
 'Argentina': {'latest_page': 166, 'status': 'DONE'},
 'Armenia': {'latest_page': 7, 'status': 'DONE'},
 'Aruba': {'latest_page': 2, 'status': 'DONE'},
 'Australia': {'latest_page': 35, 'status': 'NOT DONE'}}
+-----------------------------------------------------------------+


iterate range 100:   0%|          | 0/240 [00:00<?, ?it/s]

Australia의 35번째 page 이미지 저장 완료
Australia의 36번째 page 이미지 저장 완료
Australia의 37번째 page 이미지 저장 완료
Australia의 38번째 page 이미지 저장 완료
Australia의 39번째 page 이미지 저장 완료
Australia의 40번째 page 이미지 저장 완료
Australia의 41번째 page 이미지 저장 완료
Australia의 42번째 page 이미지 저장 완료
Australia의 43번째 page 이미지 저장 완료
[KeyboardInterrupt] 현재까지의 status :
{'Afghanistan': {'latest_page': 1, 'status': 'DONE'},
 'Albania': {'latest_page': 15, 'status': 'DONE'},
 'Algeria': {'latest_page': 9, 'status': 'DONE'},
 'American Samoa': {'latest_page': 1, 'status': 'DONE'},
 'Andorra': {'latest_page': 1, 'status': 'DONE'},
 'Angola': {'latest_page': 4, 'status': 'DONE'},
 'Anguilla': {'latest_page': 0, 'status': 'DONE'},
 'Antarctica': {'latest_page': 0, 'status': 'DONE'},
 'Antigua and Barbuda': {'latest_page': 0, 'status': 'DONE'},
 'Argentina': {'latest_page': 166, 'status': 'DONE'},
 'Armenia': {'latest_page': 7, 'status': 'DONE'},
 'Aruba': {'latest_page': 2, 'status': 'DONE'},
 'Australia': {'latest_page': 44, 'status': 'NOT DONE'}}
